# The Data Scientist's Notebook

Only data privacy is protected in this use case. The model created by the data scientist is not kept private since it is sent to the data owner's machine for training without being encrypted.

**Note:**

Much of the code used here is either copied or adapted from the `Word-level language modeling` PyTorch example:

https://github.com/pytorch/examples/tree/master/word_language_model

The goal being to demonstrate how the original example could be adapted to a context where the dataset is private to the data owner as it is the case in this demo.

## PART 0: Connect to a Remote Duet Server

In [1]:
import syft as sy
import torch

Before connecting to the remote duet server, the data owner should first launch a duet server. After launch, the data scientist can connect to the duet server.

In [2]:
duet = sy.join_duet(loopback=True)

🎤  🎸  ♪♪♪ joining duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently 
♫♫♫ > in alpha. Do not use this to protect real-world data.
♫♫♫ >
♫♫♫ > Punching through firewall to OpenGrid Network Node at: http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ > http://ec2-18-216-8-163.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... DONE!

♫♫♫ > Duet Client ID: 73f48b52b721a1b022a1d6ebe28175fc

♫♫♫ > STEP 1: Send the Duet Client ID to your duet partner!

♫♫♫ > ...waiting for partner to connect...
♫♫♫ > ...using a running event loop...

♫♫♫ > CONNECTED!


2020-11-09 09:59:09.108 | ERROR    | syft.grid.duet.webrtc_duet:pull:250 - Got an exception in Duet pull. 
2020-11-09 09:59:12.196 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-09 09:59:17.199 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-09 09:59:22.200 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-09 09:59:27.201 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


## PART 1: Get Pointers to Shared Objects

Get a list of the shared objects:

In [3]:
duet.store.pandas

,ID,Tags,Description
0,<UID:399e0394-b29f-48c8-8934-579e4a9191c8>,"[wikitext2_dataset, train_data]","Wikitext2 training set. shape: (2088628,)"
1,<UID:b91a71b6-9d48-4c2c-8e04-f85268fad043>,"[wikitext2_dataset, vocab_size]",Vocabulary size of Wikitext2 dataset


Get the size of the dataset's vocabulary

In [4]:
vocab_size = duet.store[1]
vocab_size = vocab_size.get(request_block = True)
vocab_size = int(vocab_size)
vocab_size


> Waiting for Blocking Request
<UID:d5468cb6-54eb-4a11-8f1f-05295bae7803>

> INSIDE Request BLOCK 1.1920928955078125e-06 seconds False


2020-11-08 13:41:20.650 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6


.
> INSIDE Request BLOCK 1.083735466003418 seconds False
Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 6

> Blocking Request ACCEPTED


33278

Get references to the datasets

In [9]:
train_data = duet.store[0]
#valid_set = duet.store[2]
#train_set = duet.store['22d54a82-b7e7-40da-adfb-06a588121ba1']

2020-11-08 13:42:05.672 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:42:10.675 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


## PART 2: Prepare Datasets for Training

The training and validation sets, as shared by the data owners, are flat tensors of the form:

```
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16 .....]
```

where the integers represent words.

The data scientist here has the responsiblity of batchifying this dataset to serve training. I avoid giving the DO this responsiblity as I suppose that it is up to the DS to decide how data is batchified.

I suggest to reshape in the following way:

1. Reshape into a list of input/target samples:

```
[ 
  [ [1 , 2 , 3 , 4 ],
    [5 , 6 , 7 , 8 ]  ],
    
  [ [9 , 10, 11, 12], 
    [13, 14, 15, 16]  ],
  :
  :
  :
]
```

This should use the `view()` method in `torch`.

2. Create a `Dataloader` object in the DS side that batchified this training set. For example, for a batch size of 2, the data loader should return:

```
Input batch:
     [ [1 , 2 , 3 , 4 ],
       [9 , 10, 11, 12]  ]
       
Target batch:
     [ [5 , 6 , 7 , 8 ],
       [13, 14, 15, 16]  ]
```

Of course all operations are carried out on tensor pointers because the dataset does not quit the DO's node.

### Build a Dataset class

In [10]:
ntokens = 2088628
bsz = 4
bptt = 32

2020-11-08 13:42:15.679 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:42:20.682 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [8]:
#train_data = torch.tensor(list(range(ntokens)), dtype = torch.long)
#train_data

tensor([      0,       1,       2,  ..., 2088625, 2088626, 2088627])

2020-11-08 13:41:50.665 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:41:55.668 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:42:00.670 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [11]:
# Get the number of batches that can be created from the training set
num_batches = ntokens // bsz

# remove surplus tokens
train_data = train_data.narrow(0, 0, bsz * num_batches)

# Reshape
train_data = train_data.view(bsz, -1)

2020-11-08 13:42:25.684 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:42:30.687 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [12]:
def get_batch(index):
    input = train_data.narrow(dim = 1, start = index, length = bptt)
    target = train_data.narrow(dim = 1, start = index + bptt, length = bptt)
    
    return input, target

2020-11-08 13:42:35.691 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [13]:
input, target = get_batch(4)
input.shape, target.shape

AttributeError: 'TensorPointer' object has no attribute 'shape'

2020-11-08 13:42:40.693 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:42:45.696 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:43:05.706 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:43:10.710 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [38]:
input.get(request_block=True)


> Waiting for Blocking Request
<UID:60e6b2f9-a729-4059-a8cf-337459d9c134>

> INSIDE Request BLOCK 9.5367431640625e-07 seconds False
.
> INSIDE Request BLOCK 1.0519609451293945 seconds False

> Blocking Request ACCEPTED


tensor([[    4,     1,     0,     0,     5,     6,     2,     7,     8,     9,
             3,    10,    11,     8,    12,    13,    14,    15,     2,    16,
            17,    18,     7,    19,    13,    20,    21,    22,    23,     2,
             3,     4],
        [ 3220,  2004,    13, 23079, 18202,    13,    43,    86,    15, 18202,
           131,    17,    59,   578,  2004,    22,  5935,    17,  2015,    43,
          1908,  1193,    15,     0,    83,   278,  3822,  8034,   131,   826,
         23075,    37],
        [   17,  6594,    61,    39,    27,  7615,  3840,    15,     0,     0,
             1,     1,     1,     9,    93,    83, 19900,     1,     1,     1,
             0,     0,   123,  7998,  3746,    16,    17,   545,    16,   479,
          7640,   664],
        [  131, 14584,    43,  1129,   651,    16,    17,  9170,    35,    17,
            52, 16703,  4075,    13,  1754,    17,     9,    37,     9, 30155,
            13,    46,   131,   808,   538,    43,    17,  

2020-11-07 23:43:45.631 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:43:50.632 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:46:25.712 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:46:30.715 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [10]:
train_data.shape

torch.Size([4, 250])

2020-11-07 23:12:44.629 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:12:49.631 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:18:24.817 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:18:29.820 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [95]:
class Wikitext2(torch.utils.data.Dataset):
    
    def __init__(self, tokens, ntokens, bptt = 35):
        
        # A pointer to the tensor that contains the list of 
        # all token IDs in the dataset
        self.tokens = tokens
        
        # The sequence length
        self.bptt = bptt
        
        # Number of tokens in the dataset
        self.ntokens = ntokens
        
        
    def __getitem__(self, index):
        
        # The `index` argument won't be used
        # The batch index will be randomly
        # sampled between 0 and ntokens - 2*bptt
        input_start = torch.randint(self.ntokens - 2 * self.bptt, size = (1,)).item()
        target_start = input_start + self.bptt
        
        
        # Get the input
        input_ptr = self.tokens.narrow(0, input_start, self.bptt)
        
        # Get the target
        target_ptr = self.tokens.narrow(0, target_start, self.bptt)
        
        
        return input_ptr, target_ptr

        
    def __len__(self):
        
        # Then number of possible training examples that could be
        # created is actually `ntokens - 2 * bptt`
        # However, to make a training epoch shorter for the
        # sake of this demo, I will choose a smaller number:
        
        return self.ntokens // (2 * self.bptt)
    
    def collate_samples(self, batch):
        
        inputs, targets = list(zip(*batch))

        return torch.stack(inputs), torch.stack(targets)

In [96]:
#train_set = Wikitext2(tokens = torch.tensor(list(range(1000)), dtype = torch.long), ntokens = 1000, bptt = 35)
train_set = Wikitext2(tokens = train_data, ntokens = 2088628, bptt = 35)

In [97]:
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=4,
                                          num_workers=0, drop_last=True,
                                          pin_memory=True, collate_fn=train_set.collate_samples
                                         )

In [98]:
for i, t  in train_loader:
    print(i)
    break

TypeError: expected Tensor as element 0 in argument 0, but got TensorPointer

2020-11-07 18:40:48.057 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 18:40:53.061 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 23:04:30.486 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-07 23:04:35.489 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Fix some hyperparameters:

In [39]:
# BPTT: Backprop through time. AKA, RNN depth.
bptt = 35
dropout = 0.5

ninp = 100

# Size of hidden layer
nhid = 200

# Number of RNN layer
nlayers =2

2020-11-07 23:46:35.717 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-07 23:46:40.720 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [ ]:
sq_count = 2088628 // (bptt * 2)
print(sq_count)
train_set = train_set.narrow(0,0,sq_count * 2 * bptt)
train_set = train_set.view(sq_count, 2, bptt)

In [ ]:
import torch
a = torch.cat([train_set[0][0], train_set[1][0]], dim=0)

In [ ]:
train_set = train_set.get(request_block=True)

In [ ]:
train_set.size()

In [ ]:
train_set

------ DRAFT --------------

In [ ]:
# b = b.get(request_block=True)

In [ ]:
# bptt = 6
# sq_count = 2088628 // (bptt * 2)
# print(sq_count)
# #b = a.narrow(0,0,sq_count * 2 * bptt)
# b = a.view(sq_count, 2, bptt)

In [ ]:
torch_d = duet.torch

In [ ]:
import torch
b = torch.tensor(range(1002))
b.size(0)

In [ ]:
bsz = 4
bptt = 6
sq_count = b.size(0) // (bsz * bptt)
print(sq_count)
b = b.narrow(0,0,sq_count * bsz * bptt)

In [ ]:
b.size(0)

In [ ]:
#b.view(sq_count, 2, bsz, bptt // 2)
#b = b.view(sq_count, bsz, bptt)
bptt = 6
sq_count = b.size(0) // (bptt * 2)
print(sq_count)
b = b.narrow(0,0,sq_count * 2 * bptt)
b = b.view(sq_count, 2, bptt)

In [ ]:
train_loader = torch.utils.data.DataLoader(b, batch_size=2)

In [ ]:
for batch in train_loader:
    print(batch)
    break

## PART 3: Built an RNN-based model

Get a pointer to the remote torch and its modules

In [85]:
torch_do = duet.torch
nn = torch_do.nn
F = torch_do.nn.functional

Create the model

In [93]:
class RNNModel(sy.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        #self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, x):
        input, hidden = x
        #emb = self.drop(self.encoder(input))
        emb = self.encoder(input)

        result = self.rnn(emb)
        output, hidden = result[0], result[1]
        
        #output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        #return F.log_softmax(decoded, dim=1)#, hidden
        return result

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

Create a model instance

In [94]:
model = RNNModel(rnn_type = 'LSTM', ninp = 100, ntoken = 100, nhid = 100, nlayers =2)

In [95]:
import torch

input = torch.ones(2,20, dtype = torch.long).send(duet) # bsz * bptt
hidden = torch.zeros(2, 2, 20, 100).send(duet) # nb_layers * bptt * nhid
#c = torch.zeros(2, 20, 100).send(duet) # nb_layers * bptt * nhid

In [96]:
#model((input, torch.zeros(1,20)))
output = model((input, None ))


2020-11-08 14:17:08.100 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [97]:
a = output.get(request_block=True)


> Waiting for Blocking Request
<UID:bee6204e-dd78-4285-8fd2-f3fafc816bd5>

> INSIDE Request BLOCK 9.5367431640625e-07 seconds False
.
> INSIDE Request BLOCK 1.0921542644500732 seconds False

> Blocking Request ACCEPTED


In [98]:
type(a)

syft.lib.python.list.List

2020-11-08 14:17:13.103 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [99]:
a[0]

tensor([[[-0.0508, -0.0066,  0.0321,  ..., -0.0376, -0.0143,  0.0267],
         [-0.0702, -0.0425, -0.0247,  ..., -0.0223, -0.0134,  0.0275],
         [-0.0412, -0.0219, -0.0043,  ...,  0.0132, -0.0031,  0.0227],
         ...,
         [-0.0588, -0.0437,  0.0031,  ...,  0.0028, -0.0540,  0.0208],
         [-0.0732, -0.0414, -0.0130,  ..., -0.0141, -0.0332,  0.0459],
         [-0.0805, -0.0324, -0.0187,  ..., -0.0020, -0.0506,  0.0423]],

        [[-0.0828, -0.0275,  0.0328,  ..., -0.0491, -0.0568,  0.0244],
         [-0.0856, -0.0426, -0.0334,  ..., -0.0117, -0.0426,  0.0124],
         [-0.1050, -0.0155, -0.0292,  ...,  0.0189, -0.0018,  0.0058],
         ...,
         [-0.0907, -0.0656,  0.0247,  ..., -0.0456, -0.0853,  0.0271],
         [-0.0510,  0.0182,  0.0185,  ...,  0.0043, -0.0578,  0.0180],
         [-0.0998, -0.0808,  0.0008,  ...,  0.0566, -0.0190, -0.0198]]],
       requires_grad=True)

2020-11-08 14:17:18.105 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:17:23.108 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

2020-11-08 14:19:08.159 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:19:13.161 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:19:18.163 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [67]:
l = sy.lib.python.list.List([2,3]).send(duet)

In [69]:
l[0]

2020-11-08 13:56:22.472 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:27.474 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:32.477 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:00:52.611 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:00:57.613 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [66]:
a

2

2020-11-08 13:55:27.439 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:55:32.443 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:07.464 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:56:12.466 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [43]:
a[0].shape

torch.Size([2, 20, 100])

2020-11-08 13:49:45.838 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:49:50.839 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:51:15.884 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:51:20.886 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [31]:
output

2020-11-08 13:46:54.739 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:46:59.741 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:04.742 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:09.745 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 13:47:14.747 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [ ]:
c = torch.ones(5,5)
c[torch.LongTensor([1,3])]

In [ ]:
torch.nn.init.xavier_uniform_(torch.empty(5,5), gain=1.0).send(duet)

_____ Local model test ______

In [81]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    """Container module with an encoder, a recurrent module, and a decoder."""

    def __init__(self, rnn_type, ntoken, ninp, nhid, nlayers, dropout=0.5, tie_weights=False):
        super(RNNModel, self).__init__()
        self.ntoken = ntoken
        #self.drop = nn.Dropout(dropout)
        self.encoder = nn.Embedding(ntoken, ninp)
        if rnn_type in ['LSTM', 'GRU']:
            self.rnn = getattr(nn, rnn_type)(ninp, nhid, nlayers, dropout=dropout)
        else:
            try:
                nonlinearity = {'RNN_TANH': 'tanh', 'RNN_RELU': 'relu'}[rnn_type]
            except KeyError:
                raise ValueError( """An invalid option for `--model` was supplied,
                                 options are ['LSTM', 'GRU', 'RNN_TANH' or 'RNN_RELU']""")
            self.rnn = nn.RNN(ninp, nhid, nlayers, nonlinearity=nonlinearity, dropout=dropout)
        self.decoder = nn.Linear(nhid, ntoken)

        # Optionally tie weights as in:
        # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
        # https://arxiv.org/abs/1608.05859
        # and
        # "Tying Word Vectors and Word Classifiers: A Loss Framework for Language Modeling" (Inan et al. 2016)
        # https://arxiv.org/abs/1611.01462
        if tie_weights:
            if nhid != ninp:
                raise ValueError('When using the tied flag, nhid must be equal to emsize')
            self.decoder.weight = self.encoder.weight

        #self.init_weights()

        self.rnn_type = rnn_type
        self.nhid = nhid
        self.nlayers = nlayers

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.weight)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, x):
        input, hidden = x
        #emb = self.drop(self.encoder(input))
        emb = self.encoder(input)

        output, hidden = self.rnn(emb)
        #output = self.drop(output)
        decoded = self.decoder(output)
        decoded = decoded.view(-1, self.ntoken)
        return F.log_softmax(decoded, dim=1), hidden

    def init_hidden(self, bsz):
        weight = next(self.parameters())
        if self.rnn_type == 'LSTM':
            return (weight.new_zeros(self.nlayers, bsz, self.nhid),
                    weight.new_zeros(self.nlayers, bsz, self.nhid))
        else:
            return weight.new_zeros(self.nlayers, bsz, self.nhid)

In [82]:
model = RNNModel(rnn_type = 'LSTM', ninp = 100, ntoken = 100, nhid = 100, nlayers =2)

In [83]:
input = torch.ones(2,20, dtype = torch.long) # bsz * bptt
hidden = torch.zeros(2, 20, 100) # nb_layers * bptt * nhid
c = torch.zeros(2, 20, 100) # nb_layers * bptt * nhid

2020-11-08 14:15:23.030 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:28.032 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:33.035 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:38.037 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [84]:
#output = model((input, hidden))
#output, hidden = model((input, [hidden, c]))
output= model((input, None))

/home/alan/deeplearning/openmined/syft_0.3.0/src/syft/lib/torch/uppercase_tensor.py:37: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  grad = getattr(self.value, "grad", None)
2020-11-08 14:15:43.039 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:48.041 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:53.044 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:58.046 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:16:03.048 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


### Remaining issues

1. passing tuple of (hidden, c) to self.rnn gives exception in MergeFrom() call
2. Dropout layer does not seems to be working
3. when calling `input, hidden = self.rnn(input, None)` in LSTM case, we get two many values to unpack. This works in the local torch case. To solve the problem, I called it as `input = self.rnn(input, None)`
4. We cannot index a list pointer `l = syft.lib.python.list.List([1,2]).send(duet); a = l[0]`
5. We cannot unpack a list pointer `a,b = l`

In [3]:
import torch
t1 = torch.tensor([2,3,5])
t2 = torch.tensor([4,1,6])

list_ptr = sy.lib.python.list.List([t1,t2]).tag('#list').send(duet)
elem_ptr = list_ptr[0] #Does not work

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [4]:
duet.store.pandas

""


2020-11-09 09:59:32.205 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-09 09:59:37.208 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-10 00:45:25.189 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-10 00:45:30.190 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [30]:
t = duet.torch.Tensor(elem_ptr )

2020-11-08 20:19:30.843 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [32]:
t

2020-11-08 20:21:30.903 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:21:35.906 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:21:40.909 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [16]:
t = t + 3

2020-11-08 17:39:20.178 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


In [33]:
t = t.get(request_block = True)


> Waiting for Blocking Request
<UID:ea027da3-e0ca-43dc-8a96-dccb18e9e288>

> INSIDE Request BLOCK 1.1920928955078125e-06 seconds False


2020-11-08 20:21:45.911 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


2020-11-08 20:21:50.912 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


...

2020-11-08 22:18:14.516 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


2020-11-08 22:18:19.518 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 4


KeyboardInterrupt: 

In [18]:
type(t)

NoneType

2020-11-08 17:39:46.823 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 17:39:51.826 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

2020-11-08 20:06:10.340 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:06:15.342 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


2020-11-08 20:06:20.344 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 3


In [78]:
t1_ptr, t2_ptr = list_ptr

ValueError: too many values to unpack (expected 2)

2020-11-08 14:10:42.889 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:10:47.891 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


...

Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:14:58.018 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


2020-11-08 14:15:03.020 | CRITICAL | syft.grid.connections.webrtc:heartbeat:563 - Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5


Joiner PQ: 0 / 0 - CQ: 0 / 0 - AT: 5
